In [23]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
from tg.interactors import ModelInteractor, DataInteractor
from tg.splitters import AnchoredSplitter
from tg.datasets import DatasetFactoryLookupCallback
from tg.ts_models import ModelClassLookupCallback

2022-10-23 14:23:38.974131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset_name = "AIR_PASSENGERS"
# dataset_name = "PERFECT_SINE30"

di = DataInteractor(dataset_name=dataset_name)

# model_name = "SARIMA"
# params = {"m": di.y.period}

# model_name = "NAIVE"
# params = {"constant": 0.0}

model_name = "ARIMA"
params = {}

# model_name = "RNN"
# params = {"epochs": 700, "hidden_units": 25}

y, X = di.get_data(model_name=model_name)

In [4]:
ARIMA_class = ModelClassLookupCallback("ARIMA")
ARIMA = ARIMA_class()


In [18]:
ARIMA.fit(y=y)

12

In [9]:
ARIMA.predict_residuals()[1:]

array([ -5.96009208, -12.14135289,   7.0070036 ,   3.63224617,
       -19.01405504, -11.00236861,   1.79158081,   5.29678137,
         7.54256892,   9.26173402, -15.88019733,  16.21582503,
       -11.92483494,  -4.79619132,   4.42471706,   8.61425197,
       -33.86672399,  -9.60834966,  -2.15843017,   2.5326123 ,
        11.31942034,   5.79903586, -26.22662658,   9.34591701,
        -3.1815353 , -26.27346057,  17.67489479, -21.78354014,
        -7.3183363 , -28.95951939,   6.22738762,  -5.80134491,
        17.28749629,  -4.37879215, -14.42157608,   4.3756046 ,
        -7.68655339,  -5.9629491 ,   3.70469361,  -2.73397385,
       -44.96691589,   3.13968191, -24.92708217,  26.68387364,
       -10.21680706,  13.77592444, -29.12740169,  17.88235559,
        -5.5593308 , -32.84497523,   7.32395203,  -3.84904494,
       -22.85436917, -28.77303937,  -2.56262197,  15.12645078,
         8.39574009,  17.23453433, -24.79907779,  23.32530982,
        12.11418564, -34.08708597,  18.20062716,  -9.49

In [11]:
from statsmodels.tsa.tsatools import lagmat

In [25]:
y

Month
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121
1949-06-01    135
1949-07-01    148
1949-08-01    148
1949-09-01    136
1949-10-01    119
1949-11-01    104
1949-12-01    118
1950-01-01    115
1950-02-01    126
1950-03-01    141
1950-04-01    135
1950-05-01    125
1950-06-01    149
1950-07-01    170
1950-08-01    170
1950-09-01    158
1950-10-01    133
1950-11-01    114
1950-12-01    140
1951-01-01    145
1951-02-01    150
1951-03-01    178
1951-04-01    163
1951-05-01    172
1951-06-01    178
1951-07-01    199
1951-08-01    199
1951-09-01    184
1951-10-01    162
1951-11-01    146
1951-12-01    166
1952-01-01    171
1952-02-01    180
1952-03-01    193
1952-04-01    181
1952-05-01    183
1952-06-01    218
1952-07-01    230
1952-08-01    242
1952-09-01    209
1952-10-01    191
1952-11-01    172
1952-12-01    194
1953-01-01    196
1953-02-01    196
1953-03-01    236
1953-04-01    235
1953-05-01    229
1953-06-01    243
1953-07-01    264
1953

In [82]:
first_model_residuals = ARIMA.predict_residuals()[1:]

y_residuals = lagmat(first_model_residuals, 12, 'both')
y_lagged = lagmat(y[1:], 12, 'both')

y_residuals = np.append(y_residuals, [pd.DataFrame(lagmat(first_model_residuals[1:], 12, "both")).iloc[-1].shift().fillna(first_model_residuals[-1]).values], axis=0)
y_lagged = np.append(y_lagged, [pd.DataFrame(lagmat(y[1:], 12, "both")).iloc[-1].shift().fillna(y.iloc[-1]).values], axis=0)

X = pd.DataFrame(np.hstack([y_residuals, y_lagged]))

In [84]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,-19.014055,3.632246,7.007004,-12.141353,-5.960092,115.0,118.0,104.0,119.0,136.0,148.0,148.0,135.0,121.0,129.0,132.0,118.0
1,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,-19.014055,3.632246,7.007004,-12.141353,126.0,115.0,118.0,104.0,119.0,136.0,148.0,148.0,135.0,121.0,129.0,132.0
2,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,-19.014055,3.632246,7.007004,141.0,126.0,115.0,118.0,104.0,119.0,136.0,148.0,148.0,135.0,121.0,129.0
3,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,-19.014055,3.632246,135.0,141.0,126.0,115.0,118.0,104.0,119.0,136.0,148.0,148.0,135.0,121.0
4,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,-19.014055,125.0,135.0,141.0,126.0,115.0,118.0,104.0,119.0,136.0,148.0,148.0,135.0
5,-33.866724,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,-11.002369,149.0,125.0,135.0,141.0,126.0,115.0,118.0,104.0,119.0,136.0,148.0,148.0
6,-9.608350,-33.866724,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,1.791581,170.0,149.0,125.0,135.0,141.0,126.0,115.0,118.0,104.0,119.0,136.0,148.0
7,-2.158430,-9.608350,-33.866724,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,5.296781,170.0,170.0,149.0,125.0,135.0,141.0,126.0,115.0,118.0,104.0,119.0,136.0
8,2.532612,-2.158430,-9.608350,-33.866724,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,7.542569,158.0,170.0,170.0,149.0,125.0,135.0,141.0,126.0,115.0,118.0,104.0,119.0
9,11.319420,2.532612,-2.158430,-9.608350,-33.866724,8.614252,4.424717,-4.796191,-11.924835,16.215825,-15.880197,9.261734,133.0,158.0,170.0,170.0,149.0,125.0,135.0,141.0,126.0,115.0,118.0,104.0


array([[115., 118., 104., ..., 129., 132., 118.],
       [126., 115., 118., ..., 121., 129., 132.],
       [141., 126., 115., ..., 135., 121., 129.],
       ...,
       [461., 508., 606., ..., 417., 405., 362.],
       [390., 461., 508., ..., 391., 417., 405.],
       [432., 390., 461., ..., 419., 391., 417.]])

In [29]:
pd.DataFrame(X.iloc[-1].shift(-1).fillna(
            y.iloc[-1]).values.reshape(1, -1))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-9.525317,41.230184,5.964201,-63.925712,-94.199058,30.663248,-54.662754,-6.016063,19.704658,27.429405,-33.440054,390.0,461.0,508.0,606.0,622.0,535.0,472.0,461.0,419.0,391.0,417.0,405.0,432.0


In [ ]:
arima_errors = np.array(y - arima_y_pred)[1:]


last_arima_errors = lagmat(arima_errors, self.arima_m, "both")
lagged_y = lagmat(y[1:], self.arima_m, "both")
svm_features = np.hstack([last_arima_errors, lagged_y])
svm_y_train = y[-(y_len - self.arima_m) + 1:].reshape(-1)

self.svr.fit(svm_features, svm_y_train)

last_arima_errors = lagmat(arima_errors, self.arima_m - 1, "both",
                        "in")
lagged_y = lagmat(y, self.arima_m - 1, "both", "in")

self.svr_input = np.hstack([last_arima_errors[-1],
                            lagged_y[-1]]).reshape(1, -1)

In [7]:
ARIMA.model.predict_in_sample()

1949-01-01      0.000000
1949-02-01    112.039908
1949-03-01    119.858647
1949-04-01    136.007004
1949-05-01    124.632246
                 ...    
1960-08-01    611.964201
1960-09-01    549.230184
1960-10-01    451.474683
1960-11-01    427.629163
1960-12-01    393.635883
Freq: MS, Name: predicted_mean, Length: 144, dtype: float64

In [8]:
last_arima_errors = lagmat(arima_errors, self.arima_m, "both")
lagged_y = lagmat(y[1:], self.arima_m, "both")

NameError: name 'lagmat' is not defined